# Visual Sentiment

## **Test use GPU**

Go to Runtime

select accelerator: GPU

SAVE

In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


## **Transfer files from gdrive to colaborator**

###**Google Drive Authenticate**

In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#1.Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

###**Load Session**

create directory tree

In [0]:
!mkdir data
!mkdir models
!mkdir results

Loading dataset file

In [0]:
#2. Load files by ID from Google Drive:
#  occorre caricare i file su Google Drive, poi renderli pubblici e copiare l'id del file dal loro link pubblico

#Esempio:
# Dataset.zip <=> https://drive.google.com/open?id=1kJwMvgml5haYrTpr_30n0enUMPX-UyT9

idfile="1kJwMvgml5haYrTpr_30n0enUMPX-UyT9" #
downloaded = drive.CreateFile({'id': idfile})
downloaded.GetContentFile('Dataset.zip') 


Extract tree and files of the session 

In [5]:
#Decompressione dei file caricati
!unzip Dataset.zip
#!rm Dataset.zip

Archive:  Dataset.zip
   creating: Dataset/
  inflating: dataset.txt             
  inflating: Dataset/img00001.jpg    
  inflating: Dataset/img00002.jpg    
  inflating: Dataset/img00003.jpg    
  inflating: Dataset/img00004.jpg    
  inflating: Dataset/img00005.jpg    
  inflating: Dataset/img00006.jpg    
  inflating: Dataset/img00007.jpg    
  inflating: Dataset/img00008.jpg    
  inflating: Dataset/img00009.jpg    
  inflating: Dataset/img00010.jpg    


In [0]:
#Controllo se i file sono stati caricati e decompressi correttamente:
!ls

### Resizing and Padding


In [0]:
from PIL import Image
import os
import glob

def scale(image, max_size, method=Image.ANTIALIAS):
    """
    resize 'image' to 'max_size' keeping the aspect ratio 
    and place it in center of white 'max_size' image 
    """
    im_aspect = float(image.size[0])/float(image.size[1])
    out_aspect = float(max_size[0])/float(max_size[1])
    if im_aspect >= out_aspect:
        scaled = image.resize((max_size[0], int((float(max_size[0])/im_aspect) + 0.5)), method)
    else:
        scaled = image.resize((int((float(max_size[1])*im_aspect) + 0.5), max_size[1]), method)
 
    offset = (((max_size[0] - scaled.size[0]) / 2), ((max_size[1] - scaled.size[1]) / 2))
    back = Image.new("RGB", max_size, "black")
    back.paste(scaled, offset)
    return back



path_results="resized"
if not os.path.exists(path_results):
    os.mkdir(path_results)

files=glob.glob("Dataset/*")
for i,f in enumerate(files):
    
    if i%500==0: print("{} of {}".format(i,len(files)))
    try:
        im = Image.open(f)
        max_size=max(im.size)
        #print("from {} to {}".format(im.size,(max_size,max_size)))
        scaled=scale(im,(max_size,max_size))
        resized=scaled.resize((224,224))
        resized.save(path_results+"/"+os.path.basename(f))
        #raw_input()
    except:
        print("Error: {}".format(f))
print("Done!!")

0 of 9248
500 of 9248
1000 of 9248
1500 of 9248
2000 of 9248
2500 of 9248
3000 of 9248
3500 of 9248
4000 of 9248
4500 of 9248
5000 of 9248
5500 of 9248
6000 of 9248
6500 of 9248
7000 of 9248
7500 of 9248
8000 of 9248
Error: Dataset/img03312.jpg
8500 of 9248
9000 of 9248
Done!!


In [0]:
!rm -r Dataset
!mv resized Dataset

mv: cannot stat 'resized': No such file or directory


## Workspace

###**Loading common modules**

In [6]:
import keras
#from keras.applications.vgg16 import VGG16
#from keras.applications.vgg16 import preprocess_input, decode_predictions
#from keras.applications.inception_resnet_v2 import InceptionResNetV2

from keras.preprocessing import image
from keras.utils import to_categorical
from keras.layers import *
from keras.models import Model
from keras.optimizers import SGD, Adam
from keras.callbacks import CSVLogger, ModelCheckpoint

from keras.legacy import interfaces
import keras.backend as K
from keras.optimizers import Optimizer

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from os.path import join

save_path="results/"
if not os.path.exists(save_path):
    os.makedirs(save_path)
    
basepath=save_path+"finetuning"
modello=basepath+"-e{epoch:02d}-vba{val_acc:.2f}.h5"
csv=basepath+"-finetuning.csv"
plot1=basepath+"-plot1.png"

classeMax=3  #Quante classi devo classificare


encoded_labels = to_categorical(range(classeMax)) #Codifico le label in one-hot vectors

Using TensorFlow backend.


### Data managemennt and manipolation (preprocessing)

In [0]:
def loadDataset(datasetFile):
    """
    ritorna:
    - la lista dei nomi dei file immagine
    - visual sentiment
    -..
    - la lista dei testi in chiaro contenuti nell'immagini
    
    datasetFile: percorso del file
    """
            
    datasetList= pd.read_csv(datasetFile, sep=' ', quotechar="@", header=None, names = ['imgPath', 'V', 'Tx', 'O', 'text'])
        
    return datasetList


def shuffling(datasetList, shuffle=1):
    if shuffle==1:
        datasetList= datasetList.sample(frac=1) 
        
    return datasetList   
    
        
def splitTrainValidationSet(datasetList, r, shuffle=1):
    """
    ritorna la lista dei file da destinare al training set e quelli da destinare al 
    validation set
    
    classifiedFileList: lista dei file
    r: rapporto di slitting (training/validation samples)
    shuffle: (default 1), casualizza l'ordine dei file di ogni classe prima dello split
    """
    
    datasetListShuffled= shuffling(datasetList, shuffle)

    idx = int(len(datasetListShuffled)*(1-r)) # dimensione del set di validazione per classe
    trainClassifiedFileList= datasetListShuffled[idx:]
    valClassifiedFileList= datasetListShuffled[:idx]
        
    return trainClassifiedFileList, valClassifiedFileList
    
    
def toBalance(datasetList, classificationColumn, shuffle=1):
    """
    ritorna la lista dei file bilanciata (stesso numero di file per ciascuna classe).
    Il numero dei file per ogni classe sara' uguale al numero di file della classe 
    col minor numero di elementi
    
    fileList: lista dei file
    shuffle: (default 1), casualizza l'ordine dei file di ogni classe
    """
    datasetListShuffle= shuffling(datasetList, shuffle)
    
    classLabels= np.unique(datasetListShuffle[classificationColumn].values)
    
    classFileLen=[]
    classIndexes=[]
    for i in classLabels:
        indexes= datasetListShuffle.index.values[datasetListShuffle[classificationColumn].values==i]
        classLen= len(indexes)
        classIndexes.append(indexes)
        classFileLen.append(classLen)
        
    minLen= min(classFileLen)
    
    indexes= np.ndarray((0,), 'int')
    for i in range(len(classFileLen)):
        ind= classIndexes[i][:minLen]
        indexes= np.append(indexes,ind)
        
    balancedDatasetList= datasetListShuffle.loc[indexes]
    
    return balancedDatasetList
  

def loadInputImages(fileNames, dirPath=""):
    """
    carica in memoria il file (nel formato opportuno) al percorso "fileName"
    """
    #images=np.ndarray((0, 224, 224, 3))
    images=[]
    for fileName in fileNames:
        #image= scipy.misc.imread(join(dirPath,fileName))
        img = image.load_img(join(dirPath,fileName), target_size=(224, 224),grayscale=False)
        img = image.img_to_array(img)
        #images= np.images(images, [img], axis=0)
        images.append(img)
     
    return np.array(images)


### Ausiliar Function

Definizione "nuovo" algoritmo SGD (permette Multiple Learning Rates) 

In [0]:
class LR_SGD(Optimizer):
    """Stochastic gradient descent optimizer.

    Includes support for momentum,
    learning rate decay, and Nesterov momentum.

    # Arguments
        lr: float >= 0. Learning rate.
        momentum: float >= 0. Parameter updates momentum.
        decay: float >= 0. Learning rate decay over each update.
        nesterov: boolean. Whether to apply Nesterov momentum.
    """

    def __init__(self, lr=0.01, momentum=0., decay=0.,
                 nesterov=False,multipliers=None,**kwargs):
        super(LR_SGD, self).__init__(**kwargs)
        with K.name_scope(self.__class__.__name__):
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self.lr = K.variable(lr, name='lr')
            self.momentum = K.variable(momentum, name='momentum')
            self.decay = K.variable(decay, name='decay')
        self.initial_decay = decay
        self.nesterov = nesterov
        self.lr_multipliers = multipliers

    @interfaces.legacy_get_updates_support
    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]

        lr = self.lr
        if self.initial_decay > 0:
            lr *= (1. / (1. + self.decay * K.cast(self.iterations,
                                                  K.dtype(self.decay))))
        # momentum
        shapes = [K.int_shape(p) for p in params]
        moments = [K.zeros(shape) for shape in shapes]
        self.weights = [self.iterations] + moments
        for p, g, m in zip(params, grads, moments):
            
            matched_layer = [x for x in self.lr_multipliers.keys() if x in p.name]
            if matched_layer:
                new_lr = lr * self.lr_multipliers[matched_layer[0]]
            else:
                new_lr = lr

            v = self.momentum * m - new_lr * g  # velocity
            self.updates.append(K.update(m, v))

            if self.nesterov:
                new_p = p + self.momentum * v - new_lr * g
            else:
                new_p = p + v

            # Apply constraints.
            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(K.update(p, new_p))
        return self.updates

    def get_config(self):
        config = {'lr': float(K.get_value(self.lr)),
                  'momentum': float(K.get_value(self.momentum)),
                  'decay': float(K.get_value(self.decay)),
                  'nesterov': self.nesterov}
        base_config = super(LR_SGD, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


### Settaggi iniziali

In [9]:
# DATA E PREPROCESSING
dataPath=''

# MODEL
# create the base pre-trained model
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
base_model = VGG16(weights='imagenet',include_top=False,input_shape=(224,224,3))
#from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
#base_model = ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3))
#from keras.applications.densenet import DenseNet121
#base_model = DenseNet121(weights='imagenet',include_top=False,input_shape=(224,224,3))
#from keras.applications.inception_resnet_v2 import InceptionResNetV2
#base_model = InceptionResNetV2(weights='imagenet',include_top=False,input_shape=(224,224,3))
#from keras.applications.xception import Xception
#base_model = Xception(weights='imagenet',include_top=False,input_shape=(224,224,3))
#from keras.applications.mobilenet import MobileNet
#base_model = MobileNet(weights='imagenet',include_top=False,input_shape=(224,224,3))
x = base_model.output
# Classification block
x = Flatten(name='flatten')(x)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dense(4096, activation='relu', name='fc2')(x)
predictions = Dense(classeMax, activation='softmax', name='predictions')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# loading weights from pretrained model 
#model.load_weights("results/epoc72.hdf5") #####


# Setting trainability of layers

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
  layer.trainable = True
  #print(layer.trainable)
  #print(layer.__class__.__name__)

  
# Setting the Learning rate multipliers

LR_mult_dict = {}
LR_mult_dict['block1_conv1']=0.1
LR_mult_dict['block1_conv2']=0.1
LR_mult_dict['block2_conv1']=0.2
LR_mult_dict['block2_conv2']=0.2
LR_mult_dict['block3_conv1']=0.3
LR_mult_dict['block3_conv2']=0.3
LR_mult_dict['block3_conv3']=0.3
LR_mult_dict['block4_conv1']=0.5
LR_mult_dict['block4_conv2']=0.5
LR_mult_dict['block4_conv3']=0.5
LR_mult_dict['block5_conv1']=0.7
LR_mult_dict['block5_conv2']=0.7
LR_mult_dict['block5_conv3']=0.7
LR_mult_dict['fc1']=1
LR_mult_dict['fc2']=1


# TRAINING
batch_size = 32
lossFunction='categorical_crossentropy'
base_lr = 1e-3 #0.00001
momentum = 0.9
#optimizer = SGD(lr=base_lr, decay=0.0, momentum=momentum, nesterov=True)
optimizer = LR_SGD(lr=base_lr, momentum=momentum, decay=0.0, nesterov=False,multipliers = LR_mult_dict)
epochs= 250 #2000 #250


# TEST
val_batch_size = 32
metrics=["accuracy"]







Instructions for updating:
Colocations handled automatically by placer.
58900480/58889256 [==============================] - 1s 0us/step


###**Training**

In [0]:
# LOADING TRAINING DATA E PREPROCESSING
datasetFile="dataset.txt"
datasetList= loadDataset(datasetFile)

balancedDatasetList= toBalance(datasetList, "V", shuffle=1)

r= 0.8
trainSetList, valSetList= splitTrainValidationSet(balancedDatasetList, r, shuffle=1)

print(len(trainSetList))
print(len(trainSetList[trainSetList.V==0]))
print(len(trainSetList[trainSetList.V==1]))
print(len(trainSetList[trainSetList.V==2]))

print(len(valSetList))
print(len(valSetList[valSetList.V==0]))
print(len(valSetList[valSetList.V==1]))
print(len(valSetList[valSetList.V==2]))

x_train_fileNames= trainSetList.imgPath.values #[:100]
y_train= trainSetList.V.values #[:100]

x_test_fileNames= valSetList.imgPath.values #[:100]
y_test= valSetList.V.values #[:100]

x_train= loadInputImages(x_train_fileNames, dirPath=dataPath)
y_train= to_categorical(y_train, classeMax)

print(y_train)

x_test= loadInputImages(x_test_fileNames, dirPath=dataPath)
y_test= to_categorical(y_test, classeMax)


# il dataset subisce una fase di preprocessing per poter essere utilizzato sulla rete VGG16
#x_train=preprocess_input(x_train)
#x_test=preprocess_input(x_test)


# COMPILE MODEL
model.compile(loss=lossFunction, optimizer=optimizer, metrics=metrics)


# DEFINE GENERATORS
gen = keras.preprocessing.image.ImageDataGenerator(
        rotation_range=90.,
	      horizontal_flip=True,
        vertical_flip=True,
	      featurewise_center=True,
	      featurewise_std_normalization=True
)
gen.fit(x_train)

val_gen = keras.preprocessing.image.ImageDataGenerator(
	      featurewise_center=True,
	      featurewise_std_normalization=True
)
val_gen.fit(x_test)


# CALLBACKS
csv_logger = CSVLogger(csv)
#saved_model = ModelCheckpoint(modello, save_best_only=True, save_weights_only=False, monitor="val_acc")
saved_model_best = ModelCheckpoint('results/finetuningBest.h5',save_best_only=True, save_weights_only=False, monitor="val_acc")


# TRAINING
steps= len(x_train)//batch_size
val_steps= len(x_test)//val_batch_size

#history=model.fit(x_train, y_train, batch_size=2, epochs=250, validation_data=(x_test, y_test), callbacks=[csv_logger, saved_model])
history=model.fit_generator(gen.flow(x_train, y_train, batch_size=batch_size, shuffle=True),
                    epochs=epochs, verbose=1,
                    validation_data=val_gen.flow(x_test, y_test, batch_size=val_batch_size, shuffle=False),
                    callbacks=[csv_logger, saved_model_best], #, saved_model
                    steps_per_epoch= steps, validation_steps=val_steps)

4256
1420
1423
1413
1063
353
350
360
[[0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 ...
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
Instructions for updating:
Use tf.cast instead.
Epoch 1/250
 87/133 [==================>...........] - ETA: 30s - loss: 1.0737 - acc: 0.4759

Save weights to file

In [0]:
model.save_weights("results/epoc100.h5")

plot training results

In [0]:
#Viene creato un grafico con l'andamento delle accuracy durante il finetuning:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])	#train_accuracy
plt.plot(history.history['val_acc']) #val_binary_accuracy
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
#plt.show()
plt.savefig(plot1, format='png')

**evaluation**

In [0]:
#val_gen = keras.preprocessing.image.ImageDataGenerator(
#	      featurewise_center=True,
#	      featurewise_std_normalization=True
#)
#val_gen.fit(x_test)


#model.load_weights("results/finetuning-e69-vba0.62.h5")
#res= model.evaluate_generator(val_gen.flow(x_test, y_test, batch_size=val_batch_size, shuffle=False), steps=val_steps, verbose=1)
#print(res)

model.load_weights("results/finetuningBest.h5")
res= model.evaluate_generator(val_gen.flow(x_test, y_test, batch_size=val_batch_size, shuffle=False), steps=val_steps, verbose=1)
print(res)

33/33 [==============================] - 10s 308ms/step
[1.0170776067358074, 0.5975378787878788]


In [0]:
#Controllo i risultati
!ls results

**prediction**

In [0]:
y_pred= model.predict_generator(val_gen.flow(x_test, y_test, batch_size=val_batch_size, shuffle=False), steps=val_steps, verbose=1)
print(y_pred)

33/33 [==============================] - 10s 310ms/step
[[0.05636145 0.24803051 0.695608  ]
 [0.02551727 0.62373644 0.35074627]
 [0.26109308 0.55215603 0.18675095]
 ...
 [0.03341064 0.94679487 0.01979459]
 [0.31647807 0.59580004 0.08772186]
 [0.13427217 0.24500684 0.620721  ]]


In [0]:
print(type(y_pred))
print(np.shape(y_pred))
np.save('results/pred.npy',y_pred)

<type 'numpy.ndarray'>
(1056, 3)


**evaluate prediction**

In [0]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

true= np.argmax(y_test, axis=1)[:len(y_pred)]
pred= np.argmax(y_pred, axis=1)

cm= confusion_matrix(true, pred)
cm= cm.astype('float32')
print('\nconfusion matrix:')
print(cm)

accuracy= accuracy_score(true, pred)
print('\naccuracy: '+str(accuracy))

precisionMacro= precision_score(true, pred, average='macro')   
precisionWeighted= precision_score(true, pred, average='weighted')
precision= precision_score(true, pred, average=None)

recallMacro= recall_score(true, pred, average='macro')   
recallWeighted= recall_score(true, pred, average='weighted')
recall= recall_score(true, pred, average=None)

f1Macro= f1_score(true, pred, average='macro')  
f1Weighted= f1_score(true, pred, average='weighted')
f1= f1_score(true, pred, average=None)

print('\nclassification metric:')
print('precision \t recall \t f1Score')
for p,r,f in zip(precision,recall,f1):
    print(str(p)+'\t'+str(r)+'\t'+str(f))

print('\nmacro metric:')
print('precision \t recall \t f1Score')
print(str(precisionMacro)+'\t'+str(recallMacro)+'\t'+str(f1Macro))

print('\nweighted metric:')
print('precision \t recall \t f1Score')
print(str(precisionWeighted)+'\t'+str(recallWeighted)+'\t'+str(f1Weighted))





confusion matrix:
[[224.  90.  27.]
 [ 52. 241.  61.]
 [ 69. 126. 166.]]

accuracy: 0.5975378787878788

classification metric:
precision 	 recall 	 f1Score
0.6492753623188405	0.656891495601173	0.653061224489796
0.5273522975929978	0.6807909604519774	0.594327990135635
0.6535433070866141	0.4598337950138504	0.5398373983739837

macro metric:
precision 	 recall 	 f1Score
0.6100569889994841	0.5991720836890003	0.5957422043331382

weighted metric:
precision 	 recall 	 f1Score
0.609862448633441	0.5975378787878788	0.5946659913561017


**feature extraction**

In [0]:
###
dataPath=''
datasetFile="dataset.txt"
datasetList= loadDataset(datasetFile)

xfn= datasetList.imgPath.values #[:100]
y= datasetList.O.values #[:100]

x= loadInputImages(xfn, dirPath=dataPath)
y= to_categorical(y, classeMax)
###

#model.compile(loss=lossFunction, optimizer=optimizer, metrics=metrics)

model.load_weights("results/finetuningBest.h5")

model2 = Model(inputs=base_model.input, outputs=model.get_layer("fc2").output)

X= model2.predict(x,batch_size=32)#, y, batch_size=val_batch_size, shuffle=False), steps=val_steps, verbose=1)
print(feat)

NameError: ignored

In [0]:
print(type(X))
print(np.shape(X))
np.save('results/Xvis.npy',X)

print(type(y))
print(np.shape(y))
np.save('results/y.npy',y)

<type 'numpy.ndarray'>
(9247, 4096)
<type 'numpy.ndarray'>
(9247, 3)


In [0]:
y= datasetList.O.values
print(type(y))
print(np.shape(y))
np.save('results/y.npy',y)

<type 'numpy.ndarray'>
(9247,)


In [0]:
np.savetxt('results/path.txt', xfn, fmt='%s')

## **Download results**

download diretto

In [0]:
#Come scaricare un file da Google Colab
#Esempio: scarico il grafico delle accuracy
from google.colab import files
path="results/finetuning-finetuning.csv"
files.download(path)
#NB: non sempre funziona correttamente lo scaricamento da Google Colab. Se il file è troppo grande, potrebbe capitare che il download fallisca.

**Salvataggio sessione in GDrive**

compressione

In [0]:
!zip visual.zip results/finetuning-e69-vba0.62.h5 results/finetuningBest.h5 results/finetuning-finetuning.csv

trasferimento

In [0]:
#ALTERNATIVA PER SCARICARE UN FILE: facciamo l'upload da Google Colab a Google Drive (però dovete aver fatto in precedenza l'autenticazione con PyDrive)
#Esempio: salvo il model su drive

#!rm session2.zip
#!zip session_unet0.zip data/** models/** results/**


file1 = drive.CreateFile()
file1.SetContentFile('visual.zip')
file1.Upload()

#Per esperienza, anche questo metodo a volte potrebbe non funzionare...
#In questo caso provare a rifare l'autenticazione

#**Console**


upload from pc

In [0]:
#UPLOAD DATASET FROM PC
#da ripetere sia per il file train.zip sia per il test.zip
from google.colab import files
uploaded = files.upload()


In [0]:
pwd

In [0]:
!mv models/unet1500.hdf5 models/unet50-4d.hdf5
!mv models/unetW.hdf5 models/unetBest-4d.hdf5

In [0]:
!pip list

In [0]:
!rm models/unetW.hdf5

In [0]:
!rm results/*

In [0]:
!rm -r *
!rm *

rm: cannot remove '*': No such file or directory


In [0]:
del model